In [1]:
import os
from multiprocessing.process import parent_process

from langchain.retrievers import MultiQueryRetriever
from langchain_community.callbacks.fiddler_callback import MODEL_NAME
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI

API_KEY = os.getenv("DASHSCOPE_API_KEY")
llm = ChatOpenAI(api_key=API_KEY, base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", model="qwen-max")

# 创建嵌入模型
embeddings = DashScopeEmbeddings(model="text-embedding-v2", dashscope_api_key=API_KEY)

# 加载向量数据库
vector_store = FAISS.load_local("../第二章/vector_db", embeddings, allow_dangerous_deserialization=True)

retriever = MultiQueryRetriever.from_llm(llm=llm, retriever=vector_store.as_retriever())

query = "客户经理被投诉了，投诉一次扣多少分"

documents = retriever.get_relevant_documents(query)
import pprint
# retriever
pprint.pprint(retriever)
pprint.pprint(documents)
# documents[0]
count = 0
# for document in documents:
#     count += 1
#     pprint.pprint(str(count) + document.page_content)

for i, content in enumerate(documents):
    pprint.pprint(str(count) + content.page_content)

/opt/miniconda3/envs/agent/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/var/folders/jl/4f3x_qrs18zdzt62kb6pz38c0000gn/T/ipykernel_2222/3096919073.py:21: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  documents = retriever.get_relevant_documents(query)


MultiQueryRetriever(retriever=VectorStoreRetriever(tags=['FAISS', 'DashScopeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x127085880>, search_kwargs={}), llm_chain=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an AI language model assistant. Your task is \n    to generate 3 different versions of the given user \n    question to retrieve relevant documents from a vector  database. \n    By generating multiple perspectives on the user question, \n    your goal is to help the user overcome some of the limitations \n    of distance-based similarity search. Provide these alternative \n    questions separated by newlines. Original question: {question}')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x124aa4aa0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x124bac320>, root_client=<openai.OpenAI object at 0x107d35550

In [15]:
# RAGAs评估
from langchain.document_loaders import PyPDFLoader

docs = PyPDFLoader("../浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf").load()
docs

[Document(metadata={'source': '../浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf', 'page': 0}, page_content='百度文库 - 好好学习，天天向上 \n-1 \n上海浦东发展银行西安分行 \n个金客户经理管理考核暂行办法 \n \n \n第一章  总   则 \n第一条  为保证我分行个金客户经理制的顺利实施，有效调动个\n金客户经理的积极性， 促进个金业务快速、 稳定地发展， 根据总行 《上\n海浦东发展银行个人金融营销体系建设方案（试行）》要求，特制定\n《上海浦东发展银行西安分行个金客户经理管理考核暂行办法（试\n行）》（以下简称本办法）。 \n第二条  个金客户经理系指各支行（营业部）从事个人金融产品\n营销与市场开拓，为我行个人客户提供综合银行服务的我行市场人\n员。 \n第三条  考核内容分为二大类， 即个人业绩考核、 工作质量考核。\n个人业绩包括个人资产业务、负债业务、卡业务。工作质量指个人业\n务的资产质量。 \n第四条  为规范激励规则，客户经理的技术职务和薪资实行每年\n考核浮动。客户经理的奖金实行每季度考核浮动，即客户经理按其考\n核内容得分与行员等级结合，享受对应的行员等级待遇。 \n '),
 Document(metadata={'source': '../浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf', 'page': 1}, page_content='百度文库 - 好好学习，天天向上 \n-2 \n第二章  职位设置与职责 \n第五条  个金客户经理职位设置为：客户经理助理、客户经理、\n高级客户经理、资深客户经理。 \n第六条  个金客户经理的基本职责： \n（一）  客户开发。研究客户信息、联系与选择客户、与客户建\n立相互依存、相互支持的业务往来关系，扩大业务资源，创造良好业\n绩； \n（二）业务创新与产品营销。把握市场竞争变化方向，开展市场\n与客户需求的调研，对业务产品及服务进行创新；设计客户需求的产\n品组合、制订和实施市场营销方案； \n（三）客户服务。负责我行各类表内外授信业务及中间业务的受\n理和运作，进行综合性、整体性的客户服务； \n（四）防范风险，提高收益。提升风险防范意识及能力，提高经\n营产品质量； \n（五）培养人材。在

In [16]:
import os
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.llms import Tongyi
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.vectorstores import Chroma

# 初始化大模型
DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")
llm = Tongyi(api_key=DASHSCOPE_API_KEY, model_name="qwen-max")

# 创建嵌入模型
embeddings = DashScopeEmbeddings(model="text-embedding-v3", dashscope_api_key=DASHSCOPE_API_KEY)

# 创建主文档分割器
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=512)

# 创建子文档分割器
child_splitter = RecursiveCharacterTextSplitter(chunk_size=256)

# 创建向量数据库
vectorstore = Chroma(collection_name="split_parents", embedding_function=embeddings)

# 创建内存存储对象
store = InMemoryStore()
# 创建父文档检索器
retriever = ParentDocumentRetriever(vectorstore=vectorstore, docstore=store, child_splitter=child_splitter,
                                             parent_splitter=parent_splitter, search_kwargs={"k": 2})
# 添加文档集
retriever.add_documents(docs)

In [9]:
len(list(store.yield_keys()))

11

In [17]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap
from langchain.schema.output_parser import StrOutputParser

# 创建 prompt 模板
template = """You are an assistant for question-answering tasks.
Use the following pieces of retieved context to answer the question.
If you don't know the answer,  just say that you don't know.
Use two sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

# 由模板生成 prompt
prompt = ChatPromptTemplate.from_template(template)

# 创建 chain
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | llm | StrOutputParser()

query = "客户经理被投诉了，投诉一次扣多少分？"
response = chain.invoke({"question": query})
print(response)

客户经理被投诉一次扣2分。  
依据是《浦发上海浦东发展银行西安分行个金客户经理考核办法》中规定：有客户投诉的，每投诉一次扣2分。


In [18]:
from datasets import Dataset

questions = [
    "客户经理被投诉了，投诉一次扣多少分？",
    "客户经理每年评聘申报时间是怎样的？",
    "客户经理在工作中有不廉洁自律情况的，发现一次扣多少分？",
    "客户经理不服从支行工作安排，每次扣多少分？",
    "客户经理需要什么学历和工作经验才能入职？",
    "个金客户经理职位设置有哪些？"
]

ground_truths = [
    "每投诉一次扣2分",
    "每年一月份为客户经理评聘的申报时间",
    "在工作中有不廉洁自律情况的每发现一次扣50分",
    "不服从支行工作安排，每次扣2分",
    "须具备大专以上学历，至少二年以上银行工作经验",
    "个金客户经理职位设置为：客户经理助理、客户经理、高级客户经理、资深客户经理"
]

answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke({"question": query}))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

data = {
    "user_input": questions,
    "response": answers,
    "retrieved_contexts": contexts,
    "reference": ground_truths
}

dataset = Dataset.from_dict(data)
dataset


Dataset({
    features: ['user_input', 'response', 'retrieved_contexts', 'reference'],
    num_rows: 6
})

In [24]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

# 使用相同的LLM进行评估
result = evaluate(
    dataset = dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
    llm=llm,
    embeddings=embeddings
)

df = result.to_pandas()
df

Evaluating: 100%|██████████| 24/24 [00:12<00:00,  1.96it/s]


,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,客户经理被投诉了，投诉一次扣多少分？,[百度文库 - 好好学习，天天向上 \n-5 \n第五章 工作质量考核标准 \n第九条 ...,客户经理被投诉一次扣2分。 \n依据是《浦发上海浦东发展银行西安分行个金客户经理考核办法》...,每投诉一次扣2分,1.0,1.0,0.5,0.992074
1,客户经理每年评聘申报时间是怎样的？,[百度文库 - 好好学习，天天向上 \n-6 \nC.发生逾期超过3 个月，无论金额大小和笔...,客户经理每年的评聘申报时间为一月份。申报后由分行在二月份组织统一资格考试。,每年一月份为客户经理评聘的申报时间,1.0,1.0,1.0,0.935945
2,客户经理在工作中有不廉洁自律情况的，发现一次扣多少分？,[百度文库 - 好好学习，天天向上 \n-5 \n第五章 工作质量考核标准 \n第九条 ...,客户经理在工作中有不廉洁自律情况的，每发现一次扣50分。,在工作中有不廉洁自律情况的每发现一次扣50分,1.0,1.0,1.0,0.912716
3,客户经理不服从支行工作安排，每次扣多少分？,[百度文库 - 好好学习，天天向上 \n-5 \n第五章 工作质量考核标准 \n第九条 ...,客户经理不服从支行工作安排，每次扣2分。 \n不认真参加分（支）行宣传活动的，每次也扣2分。,不服从支行工作安排，每次扣2分,1.0,1.0,1.0,0.849680
4,客户经理需要什么学历和工作经验才能入职？,[],您提供的上下文中没有包含关于客户经理入职所需的学历和工作经验的信息。因此，我无法回答该问题。,须具备大专以上学历，至少二年以上银行工作经验,0.0,0.0,1.0,0.000000
5,个金客户经理职位设置有哪些？,[],我不知道个金客户经理职位设置的具体内容。 \n因为没有提供相关上下文信息。,个金客户经理职位设置为：客户经理助理、客户经理、高级客户经理、资深客户经理,0.0,0.0,1.0,0.000000
